In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [ ]:
pip install git+https://github.com/XPixelGroup/BasicSR.git

In [ ]:
from basicsr.archs.rrdbnet_arch import RRDBNet

In [ ]:
pip install einops timm opencv-python pytorch-lightning

## Importing Required Libraries

In [ ]:
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import ToTensor
from torch import nn, optim
from skimage.metrics import peak_signal_noise_ratio as psnr
from basicsr.archs.rrdbnet_arch import RRDBNet

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Define Custom Dataset for Paired Image Super-Resolution

In [ ]:
class PairedImageDataset(Dataset):
    def __init__(self, low_dir, high_dir):
        self.low_dir = low_dir
        self.high_dir = high_dir
        self.filenames = sorted(os.listdir(low_dir))

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        low = cv2.imread(os.path.join(self.low_dir, filename))[:, :, ::-1]
        high = cv2.imread(os.path.join(self.high_dir, filename))[:, :, ::-1]
        low = (low / 255.0).astype(np.float32)
        high = (high / 255.0).astype(np.float32)
        return ToTensor()(low), ToTensor()(high)




In [ ]:
# from torch.utils.data import Dataset
# from torchvision.transforms import ToTensor
# import cv2


# class PairedImageDataset(Dataset):
#     def __init__(self, low_dir, high_dir):
#         self.low_dir = low_dir
#         self.high_dir = high_dir
#         self.filenames = sorted(os.listdir(low_dir))

#     def __len__(self):
#         return len(self.filenames)

#     def __getitem__(self, idx):
#         filename = self.filenames[idx]

#         low_path = os.path.join(self.low_dir, filename)
#         high_path = os.path.join(self.high_dir, filename)

#         # Load with cv2 and ensure it loaded correctly
#         low = cv2.imread(low_path)
#         high = cv2.imread(high_path)

#         if low is None or high is None:
#             raise FileNotFoundError(f"Missing file: {filename}")

#         # Convert BGR to RGB
#         low = cv2.cvtColor(low, cv2.COLOR_BGR2RGB)
#         high = cv2.cvtColor(high, cv2.COLOR_BGR2RGB)

#         # Normalize to [0, 1]
#         low = (low / 255.0).astype(np.float32)
#         high = (high / 255.0).astype(np.float32)

#         # Convert to torch.Tensor
#         return ToTensor()(low), ToTensor()(high)


In [ ]:
# import torchvision.transforms as T
# from torchvision.transforms import ToTensor, ToPILImage

# # Define your transform pipeline
# transform = T.Compose([
#     T.ToPILImage(),
#     T.RandomHorizontalFlip(),
#     T.RandomVerticalFlip(),
#     T.RandomRotation(10),
#     T.ToTensor()
# ])

# class PairedImageDataset(Dataset):
#     def __init__(self, low_dir, high_dir, transform=None):
#         self.low_dir = low_dir
#         self.high_dir = high_dir
#         self.filenames = sorted(os.listdir(low_dir))
#         self.transform = transform

#     def __len__(self):
#         return len(self.filenames)

#     def __getitem__(self, idx):
#         filename = self.filenames[idx]
        
#         # Load and normalize
#         low = cv2.imread(os.path.join(self.low_dir, filename))[:, :, ::-1]
#         high = cv2.imread(os.path.join(self.high_dir, filename))[:, :, ::-1]

#         low = (low * 255.0).clip(0, 255).astype(np.uint8)  # Back to uint8 for PIL
#         high = (high * 255.0).clip(0, 255).astype(np.uint8)

#         if self.transform:
#             # Ensure both images undergo the same random transformation
#             seed = np.random.randint(99999)
#             torch.manual_seed(seed)
#             low = self.transform(low)
#             torch.manual_seed(seed)
#             high = self.transform(high)
#         else:
#             low = ToTensor()(low.astype(np.float32) / 255.0)
#             high = ToTensor()(high.astype(np.float32) / 255.0)

#         return low, high


### Loading Pretrained Real-ESRGAN Model

In [ ]:
def load_esrgan_model():
    model_path = '/kaggle/input/real-esrgan-model/RealESRGAN_x4plus.pth'
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32)
    state_dict = torch.load(model_path, map_location='cpu')
    if 'params_ema' in state_dict:
        state_dict = state_dict['params_ema']
    model.load_state_dict(state_dict, strict=True)
    torch.cuda.empty_cache()
    model.to(device).eval()
    return model

###  Preprocessing and Inference Image Conversion

In [ ]:
# Function to upscale a single image using the model

def process(model, img):
    # Normalize image to [0, 1]
    img = (img / 255.0).astype(np.float32)

    # Convert image to tensor and add batch dimension
    img_tensor = ToTensor()(img).unsqueeze(0).to(device)

    # Forward pass through the model
    with torch.no_grad():
        sr = model(img_tensor).clamp(0, 1)

    # Convert model output to numpy array and denormalize to [0, 255]
    sr_img = sr.squeeze().permute(1, 2, 0).cpu().numpy()
    return (sr_img * 255.0).round().astype(np.uint8)

### Evaluation Function

In [ ]:
# Evaluate model using PSNR on validation set
def evaluate(model, val_low_dir, val_high_dir):
    scores = []
    model.eval()
    for filename in os.listdir(val_low_dir):
        low = cv2.imread(os.path.join(val_low_dir, filename))[:, :, ::-1]
        high = cv2.imread(os.path.join(val_high_dir, filename))[:, :, ::-1]
        output = process(model, low)
        scores.append(psnr(high, output))

    # Return average PSNR score
    return np.mean(scores)




## Inference on Test Images

In [ ]:
# Inference function for test images
def infer(model, test_dir, save_dir):
    # Ensure output directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Set the model to evaluation mode 
    model.eval()

    
    for filename in os.listdir(test_dir):
        img = cv2.imread(os.path.join(test_dir, filename))[:, :, ::-1]
        sr_img = process(model, img)
        cv2.imwrite(os.path.join(save_dir, filename), sr_img[:, :, ::-1])


## Training Function (Fine-Tuning)

In [ ]:
def train(model, train_loader, epochs=10, lr=1e-4):

    # Set model to training mode
    model.train()

    # defining mean square error as the loss function
    criterion = nn.MSELoss()

    # use of adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # train for the specified num of epochs
    for epoch in range(epochs):

        # progress bar for tracking status
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        # Iterate through each batch
        for lr_img, hr_img in pbar:
            
            lr_img, hr_img = lr_img.to(device), hr_img.to(device)

            # clear gradiaent before bp
            optimizer.zero_grad()

            # forword pass
            sr = model(lr_img)
            loss = criterion(sr, hr_img)
            loss.backward()

            # update model parameters
            optimizer.step()

            # show current batch
            pbar.set_postfix(loss=loss.item())

        # clear cache memory to avoid mem. overflow
        torch.cuda.empty_cache()

In [ ]:
# class Discriminator(nn.Module):
#     def __init__(self):
#         super(Discriminator, self).__init__()
#         self.model = nn.Sequential(
#             nn.Conv2d(3, 64, 3, stride=2, padding=1),     # 🔁 Changed from 1 to 3
#             nn.LeakyReLU(0.2, inplace=True),

#             nn.Conv2d(64, 128, 3, stride=2, padding=1),
#             nn.BatchNorm2d(128),
#             nn.LeakyReLU(0.2, inplace=True),

#             nn.Conv2d(128, 256, 3, stride=2, padding=1),
#             nn.BatchNorm2d(256),
#             nn.LeakyReLU(0.2, inplace=True),

#             nn.Conv2d(256, 512, 3, stride=2, padding=1),
#             nn.BatchNorm2d(512),
#             nn.LeakyReLU(0.2, inplace=True),

#             nn.AdaptiveAvgPool2d((1, 1)),
#             nn.Flatten(),
#             nn.Linear(512, 1),
#             nn.Sigmoid()
#         )

#     def forward(self, x):
#         return self.model(x)


In [ ]:
import torch.nn as nn
import torch.optim as optim
import gc
from tqdm import tqdm

# def train_with_gan(model, discriminator, train_loader, epochs=2, lr=1e-4, gan_weight=1e-3):
#     model.train()
#     discriminator.train()

#     # Loss functions
#     mse_loss = nn.MSELoss()
#     gan_loss = nn.BCEWithLogitsLoss()

#     # Optimizers
#     optimizer_G = optim.Adam(model.parameters(), lr=lr)
#     optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

#     for epoch in range(epochs):
#         pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
#         for lr_img, hr_img in pbar:
#             lr_img, hr_img = lr_img.to(device), hr_img.to(device)

#             valid = torch.ones((lr_img.size(0), 1), device=device)
#             fake = torch.zeros((lr_img.size(0), 1), device=device)

#             # ---- Train Generator ----
#             optimizer_G.zero_grad()
#             sr = model(lr_img)
#             pred_fake = discriminator(sr)

#             loss_mse = mse_loss(sr, hr_img)
#             loss_gan = gan_loss(pred_fake, valid)
#             loss_G = loss_mse + gan_weight * loss_gan
#             loss_G.backward()
#             optimizer_G.step()

#             # ---- Train Discriminator ----
#             optimizer_D.zero_grad()
#             pred_real = discriminator(hr_img.detach())
#             pred_fake = discriminator(sr.detach())
#             loss_real = gan_loss(pred_real, valid)
#             loss_fake = gan_loss(pred_fake, fake)
#             loss_D = 0.5 * (loss_real + loss_fake)
#             loss_D.backward()
#             optimizer_D.step()

        #     pbar.set_postfix(MSE=loss_mse.item(), GAN=loss_gan.item())

        #     # Free memory
        #     del sr, pred_fake, pred_real, loss_G, loss_D, loss_mse, loss_gan
        #     torch.cuda.empty_cache()
        #     gc.collect()

        # torch.cuda.empty_cache()


## Main Execution Block

In [ ]:
if __name__ == "__main__":
    # Load model
    model = load_esrgan_model()
    # discriminator = Discriminator().to(device)


    # Prepare Dataset
    train_dataset = PairedImageDataset(
        '/kaggle/input/dlp-jan-2025-nppe-3/archive/train/train',
        '/kaggle/input/dlp-jan-2025-nppe-3/archive/train/gt'
        # transform=transform
    )

    # Use only half data for training
    # half_size = len(train_dataset) // 2
    # train_subset, _ = random_split(train_dataset, [half_size, len(train_dataset) - half_size])
    # train_loader = DataLoader(train_subset, batch_size=1, shuffle=True)

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

    # Fine-tune the model
    print("Starting fine-tuning...")
    train(model, train_loader, epochs=10, lr=1e-4)
    # train_with_gan(model, train_loader, epochs=1, lr=1e-3)
    # train_with_gan(model, discriminator, train_loader, epochs=2, lr=1e-4)

    # # Evaluate after fine-tuning
    # print("Evaluating on validation set...")
    # val_psnr = evaluate(model,
    #     '/kaggle/input/dlp-jan-2025-nppe-3/archive/val/val',
    #     '/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt'
    # )
    # print(f"Validation PSNR: {val_psnr:.2f}")

    # Inference on test set
    print("Running inference on test set...")
    infer(model, '/kaggle/input/dlp-jan-2025-nppe-3/archive/test', '/kaggle/working/esrgan_outputs')


# Submission

In [ ]:
# infer(model, '/kaggle/input/dlp-jan-2025-nppe-3/archive/test', '/kaggle/working/esrgan_outputs')


In [ ]:
from PIL import Image

def images_to_csv(folder_path, output_csv):
    data_rows = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L') 
            image_array = np.array(image).flatten()[::8]
            # Replace 'test_' with 'gt_' in the ID
            image_id = filename.split('.')[0].replace('test_', 'gt_')
            data_rows.append([image_id, *image_array])
    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f'Successfully saved to {output_csv}')

folder_path = '/kaggle/working/esrgan_outputs'
output_csv = 'submission.csv'
images_to_csv(folder_path, output_csv)